<a href="https://colab.research.google.com/github/svanalex/HaikusFromPaintings/blob/main/SelfEvalToolsV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Basic syllable counter

In [ ]:
import re

In [ ]:
#building a better syllable counter
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
d = cmudict.dict()

In [ ]:
import nltk
from nltk.corpus import cmudict

# Load the CMU Pronouncing Dictionary
nltk.download('cmudict')
cmu_dict = cmudict.dict()

def estimate_syllables(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word and word[0] in vowels:
        count += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def syllable_count(word):
    word = word.lower()
    if word in cmu_dict:
        # Take the first pronunciation
        pronunciation = cmu_dict[word][0]
        syllables = len([ph for ph in pronunciation if ph[-1].isdigit()])
        return syllables
    else:
        return estimate_syllables(word)


In [ ]:
def preprocess(sentence):
  sentence = re.sub('[^A-Za-z0-9 ]+', '', sentence)
  return sentence

In [ ]:
def line_counter(sentence):
  #need to make words from sentences
  sentence = preprocess(sentence)
  #print(sentence)
  sentence_syllables = 0
  words = sentence.split()
  for word in words:
    #print(word, syllable_count(word))
    sentence_syllables += syllable_count(word)
    #sentence_syllables += count_syllables(word)
  return sentence_syllables

##Checking for haikus using syllable counter?

In [ ]:
def syll_sentence(sentence):
  syll_line = []
  for line in sentence.split("\n"):
    print(line_counter(line))
    syll_line.append((line_counter(line), line))
  return syll_line

In [ ]:
def extract_flexible_haikus(syll_lines):
    haikus = []

    for i in range(1, len(syll_lines) - 1):
        prev_syll, prev_line = syll_lines[i - 1]
        curr_syll, curr_line = syll_lines[i]
        next_syll, next_line = syll_lines[i + 1]

        if 4 <= prev_syll <= 6 and 6 <= curr_syll <= 8 and 4 <= next_syll <= 6:
            haikus.append((f"{prev_line}\n{curr_line}\n{next_line}", [prev_syll, curr_syll, next_syll]))
            break
    return haikus if haikus else [("no/improper haiku", [0,0,0])]

In [ ]:
def score_haiku(haiku):
  score = sum([x-y for x,y in zip([5,7,5], haiku[1])])
  return score

In [ ]:
test_sentence = ".\n Here's the haiku:\nMoonlight whispers\nSilent brush strokes paint\nDreaming midnight blue\nThis haiku captures the essence of the painting's mood and style"
#test_sentence = ""
sen = syll_sentence(test_sentence)
haiku = extract_flexible_haikus(sen)
print(haiku)
print(haiku[0][0])
print(score_haiku(haiku[0]))

##Llama from hugging face as an eval tool

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
#text = f"How would you rate this haiku on a scale from 1-10?: {haiku[0][0]}"
text = [f"I need to grade this haiku for syllable count on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for its relation to nature on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for the quality of the creativity on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for the imagery contained within it on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}"
        ]


for line in text:
  inputs = tokenizer(line, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=80)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

##BLIP as a self evaluation mechanism

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForImageTextRetrieval

model = BlipForImageTextRetrieval.from_pretrained("Salesforce/blip-itm-base-coco")
processor = AutoProcessor.from_pretrained("Salesforce/blip-itm-base-coco")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "an image of a dog"

inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
#Sampled several HAIKU outputs for evaluation of iamge-haiku pairs. Have decided this is not an appropriate path forwards given incredibly low accuracy
image = Image.open("/content/ce8b9ee46fa699101c2d31f2b4a9622e.jpg")
#text = "painting of the moon and stars with a cityscape background"
#text = "Golden dusk unfolds Shadows climb the quiet hills Night hums soft and slow" #0.001
#text = "Black church towers rise Moonlight bleeds across the sky Sorrow echoes" #0.222
#text = "A church stands stark Where moonlight bleeds soft shadows Silence whispers" #0.002
text = "Lonely church stands tall Nocturnal shadows embrace Silence surrounds all" #0.001
text = "Stark shadows drift Lights glimmer through bare branches Silence whispers cold" #0.000
text = "Cherry tree's shadows Lights flicker between branches Silence whispers" #0.000


In [ ]:
from transformers import BlipProcessor, BlipForImageTextRetrieval
import torch

# choose the ITM‑fine‑tuned checkpoint
checkpoint = "Salesforce/blip-itm-base-coco"
processor  = BlipProcessor.from_pretrained(checkpoint)
model      = BlipForImageTextRetrieval.from_pretrained(checkpoint)

# prepare your single image + single text
inputs = processor(images=image, text=text, return_tensors="pt")

# forward pass: itm_logits has shape [1, 2]
itm_out = model(**inputs)
itm_logits = itm_out.itm_score

# turn into probabilities [P(no‑match), P(match)]
probs = itm_logits.softmax(dim=-1)

match_prob = probs[0, 1].item()
print(f"P(image matches text) = {match_prob:.3f}")



In [ ]:
image

##Trying other stuff with BLIP


In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install --upgrade numpy scipy

In [ ]:
#!pip install "transformers==4.38.2" "numpy==1.23.5"

!pip install numpy==1.25.2 transformers==4.51.3


In [ ]:
import torch
from PIL import Image
import requests
from transformers import AutoProcessor, Blip2ForImageTextRetrieval

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Blip2ForImageTextRetrieval.from_pretrained("Salesforce/blip2-itm-vit-g")
processor = AutoProcessor.from_pretrained("Salesforce/blip2-itm-vit-g")

model.to(device)
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open("/content/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg")
text = "Lonely church stands tall Nocturnal shadows embrace Silence surrounds all"

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
itm_out = model(**inputs, use_image_text_matching_head=True)
logits_per_image = torch.nn.functional.softmax(itm_out.logits_per_image, dim=1)
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

print(f"{probs[0][0]:.1%} that image 0 is not '{text}'")
print(f"{probs[0][1]:.1%} that image 0 is '{text}'")

Note: nearly all haiku outputs are graded at ~26.9% accuracy. I have deemed that BLIP is fully inappropriate for this context due to this (at least given zero shot learning)

##Llama to text multiple haikus
basically filtering 5 haikus to see if Llama can pick out a favorite/best haiku

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Llama-4
#Note, this block remains completely unused as I was unable to get access to Llama 4
from transformers import AutoProcessor, Llama4ForConditionalGeneration
import torch

model_id = "meta-llama/Llama-4-Scout-17B-16E-Instruct"

processor = AutoProcessor.from_pretrained(model_id)
model = Llama4ForConditionalGeneration.from_pretrained(
    model_id,
    attn_implementation="flex_attention",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

url1 = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
url2 = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/datasets/cat_style_layout.png"
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": url1},
            {"type": "image", "url": url2},
            {"type": "text", "text": "Can you describe how these two images are similar, and how they differ?"},
        ]
    },
]

inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
)

response = processor.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])[0]
print(response)
print(outputs[0])

In [ ]:
#text = "painting of the moon and stars with a cityscape background"
#text = "Golden dusk unfolds Shadows climb the quiet hills Night hums soft and slow" #0.001
#text = "Black church towers rise Moonlight bleeds across the sky Sorrow echoes" #0.222
#text = "A church stands stark Where moonlight bleeds soft shadows Silence whispers" #0.002
text = "Lonely church stands tall Nocturnal shadows embrace Silence surrounds all" #0.001
text = "Stark shadows drift Lights glimmer through bare branches Silence whispers cold" #0.000
text = "Cherry tree's shadows Lights flicker between branches Silence whispers" #0.000

In [ ]:
#LLama-3 70B
#Model has OOM issues with 15GB of VRAM and 51GB of DRAM
import transformers
import torch

model_id = "meta-llama/Llama-3.3-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a professional haiku judge at a competition, who is tasked with evaluating three haikus and choosing a winner."},
    {"role": "user", "content": "A church stands stark Where moonlight bleeds soft shadows Silence whispers. Lonely church stands tall Nocturnal shadows embrace Silence surrounds all. Stark shadows drift Lights glimmer through bare branches Silence whispers cold."},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

#Okay, working implementation
Llama 3.1-8B-instruct

Powerful model for prompt engineering allowing for the system to be preprompted before a contextual user prompt is provided.

This allows us to directly frame the model as a professional competition judge, and pass along multiple haikus, allowing the model to essentially self evaluate and choose a "best" haiku

In [ ]:
!nvcc --version

In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

import torch

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)
input_text = "What are we having for dinner?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
#Want some basic evaluation. Capacity of the code to support realtime users is important!
import time

In [ ]:
#Basic Evaluation Using Prompt Engineering
t = time.time()
messages = [
    {"role": "system", "content": "You are a professional haiku judge at a competition, who is tasked with evaluating three haikus and choosing a winner."},
    {"role": "user", "content": "A church stands stark Where moonlight bleeds soft shadows Silence whispers. Lonely church stands tall Nocturnal shadows embrace Silence surrounds all. Stark shadows drift Lights glimmer through bare branches Silence whispers cold."},
]

input_ids = tokenizer.encode(messages[1]['content'], return_tensors="pt").to("cuda")

outputs = quantized_model.generate(input_ids, max_new_tokens=128)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
#print(outputs[0]["generated_text"][-1])
print(f"Time taken {time.time() - t}")

In [ ]:
#Much more involved prompt engineering done below
t = time.time()
messages2 = [
    {
        "role": "system",
        "content": (
            "You are a professional haiku competition judge. "
            "Your job is to evaluate three haikus based on their poetic quality, emotional impact, imagery, and adherence to the traditional haiku structure. "
            "After evaluating, choose the best haiku and explain your reasoning in 2-3 sentences."
        )
    },
    {
        "role": "user",
        "content": (
            "Here are three haikus:\n\n"
            "1. A church stands stark\n"
            "   Where moonlight bleeds soft shadows\n"
            "   Silence whispers.\n\n"
            "2. Lonely church stands tall\n"
            "   Nocturnal shadows embrace\n"
            "   Silence surrounds all.\n\n"
            "3. Stark shadows drift\n"
            "   Lights glimmer through bare branches\n"
            "   Silence whispers cold.\n\n"
            "Please select the best one and explain why."
        )
    }
]

input_ids2 = tokenizer.encode(messages2[1]['content'], return_tensors="pt").to("cuda")

outputs2 = quantized_model.generate(input_ids2, max_new_tokens=128)

print(tokenizer.decode(outputs2[0], skip_special_tokens=True))
print(f"time taken {time.time() - t}")